In [1]:
# Volume fluxes for the different layers
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [2]:
sd = 6 # sill depth

state01 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/Qsg500/state_' + str(format(sd,'03d')) + '.nc')
grid = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/Qsg500/grid_' + str(format(sd,'03d')) + '.nc')

state1 = state01.isel(X=range(120), Xp1=range(121), Y=range(30,39), Yp1=range(30,40), T=range(89,121))

HFacW = grid.HFacW.data[:,30:39,:] # vertical fraction of open cell at West face
rA = grid.rA.data[30:39,:] # r face area at U point


Area = np.empty([50, 9]) # Cross Y direction
Area[:10, :] = 100
Area[10:20, :] = 200
Area[20:30, :] = 300
Area[30:40, :] = 800
Area[40:, :] = 1600

rho = 999.8 # Density
f = 5
s = 97


In [3]:
# Read data in T,Z,Y,X
u = state1.U.data
#u = 0.5 * (u0[:,:,:,:-1] + u0[:,:,:,1:])
uzx = u.mean(axis=(0,2)) # Z, X
w = state1.W.data
#w = 0.5 * (w0[:,:-1,:,:] + w0[:,1:,:,:])

In [4]:
# Advective flux of potential temperature
Vf = np.sum(Area * u[:,:,:,f].mean(0))
Vs = np.sum(Area * u[:,:,:,s].mean(0))

Vf, Vs

(500.0899390608538, 499.93126089684665)

In [5]:
# Find the interface and vertical flux of theta
def Layer(uzx):
    # Layer interface, zero velocity crossing point
    
    wt = w.mean(0) # Z, Y, X
    X = uzx.shape[1]
    Vzx = np.empty(X)
    itf = np.empty(X, dtype=int)
    Vzx[[0,1,-2,-1]] = 0
    itf[[0,1,-2,-1]] = 0
    for i in range(2,X-2):
        l = np.where(uzx[:, i] < 0)[-1][0]
        itf[i] = l
        Vzx[i] = np.sum(wt[l,:,i] * rA[:,i]) # Vertical advective flux through the interface
    return (itf, Vzx)

In [6]:
itf, Vzx = Layer(uzx)

#f = 5 # Control volume start index near the ice front 3
#s = 88 # Control volume end index at the sill crest 28

Vz = Vzx[f:s].sum() 

Vf1 = np.sum(HFacW[:itf[f],:,f] * Area[:itf[f],:] * u[:,:itf[f],:,f].mean(0)) 
Vs1 = np.sum(HFacW[:itf[s],:,s] * Area[:itf[s],:] * u[:,:itf[s],:,s].mean(0)) 

Vf2 = np.sum(HFacW[itf[f]:,:,f] * Area[itf[f]:,:] * u[:,itf[f]:,:,f].mean(0))
Vs2 = np.sum(HFacW[itf[s]:,:,s] * Area[itf[s]:,:] * u[:,itf[s]:,:,s].mean(0))

Vf1, Vs1, Vf2, Vs2

(3382.8672943636775,
 3808.782286569476,
 -2882.7773553028237,
 -3308.8510256726295)

In [7]:
Vzx[f:s].sum()

425.85326037608115